In [1]:
from qadataset import QADataset
from config import dataset_config, model_config

train_dataset = QADataset(dataset_config, split="train[:6]")
val_dataset = QADataset(dataset_config, split="train[:6]")

/home/atomwalk12/Dropbox (Old)/notes/vision/project/BeyondVisionQA/model.py:128: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.metric = load_metric("bertscore")
/home/atomwalk12/anaconda3/envs/questllama/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/38575 [00:00<?, ? examples/s]

Map:   0%|          | 0/38575 [00:00<?, ? examples/s]

In [2]:
# from model import get_model

# model = get_model(quantization='8bit')
from torch.utils.data import Dataset, DataLoader
from trainer_blip2 import BLIP2ModelPLModule
from trainer_blip2 import BLIP2PLModule
import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoProcessor, Blip2ForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)
# Let's define the LoraConfig
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()


train_dataloader = DataLoader(train_dataset, collate_fn=BLIP2PLModule.train_textual_labels, batch_size=8, shuffle=True, num_workers=4)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

trainable params: 5,242,880 || all params: 3,749,922,816 || trainable%: 0.1398


PeftModel(
  (base_model): LoraModel(
    (model): Blip2ForConditionalGeneration(
      (vision_model): Blip2VisionModel(
        (embeddings): Blip2VisionEmbeddings(
          (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
        )
        (encoder): Blip2Encoder(
          (layers): ModuleList(
            (0-38): 39 x Blip2EncoderLayer(
              (self_attn): Blip2Attention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): Linear8bitLt(in_features=1408, out_features=4224, bias=True)
                (projection): Linear8bitLt(in_features=1408, out_features=1408, bias=True)
              )
              (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
              (mlp): Blip2MLP(
                (activation_fn): GELUActivation()
                (fc1): Linear8bitLt(in_features=1408, out_features=6144, bias=True)
                (fc2): Linear8bitLt(in_features=6144, out_features=1408, bias=True)
      

In [3]:
from transformers import TrainingArguments, Trainer, AutoTokenizer
from dataset_configs.easy_vqa import translate
from dataset_configs.easy_vqa import filter, get_image
from PIL import Image

training_args = TrainingArguments(output_dir="blip2-easyvqa-hf_trainer", eval_strategy="epoch", )
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

train_dataset[0]
def tokenize_function(examples):
    texts = []
    images = []
    for i in range(len(examples['question'])):  # Iterate over the indices
        item = {
            'question': examples['question'][i],
            'answer': examples['answer'][i],
            'id': examples['id'][i],
            'label': examples['label'][i],
            'path': examples['path'][i]
        }
        texts.append(translate(item, training=True))
        images.append(Image.open(examples['path'][i]))
    return processor(images=images, text=texts, padding=True, return_tensors="pt")

tokenized_datasets = train_dataset.dataset.map(tokenize_function, batched=True)
tokenized_datasets = val_dataset.dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

: 

In [1]:
from model import BertScoreMetric
import numpy as np

metric = BertScoreMetric()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, lang='en')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

ImportError: cannot import name 'WUPMeasure' from partially initialized module 'model' (most likely due to a circular import) (/home/atomwalk12/Dropbox (Old)/notes/vision/project/BeyondVisionQA/model.py)

: 

In [ ]:
for epoch in range(200):
  print("Epoch:", epoch)
  for idx, inputs in enumerate(train_dataloader):
    input, labels = inputs
    
    outputs = model(**input,
                labels=labels)
    
    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

# Inference

In [ ]:
# load image
example = train_dataset[5]
image = example[0]

text_inputs = processor.tokenizer(
    example[1]["question"], padding=True, return_tensors="pt"
)
question = example[1]['question']

text = f"Question: {question} Answer:"
print(text)
image

In [ ]:
# prepare image for the model
inputs = processor(images=image, text=text, return_tensors="pt").to(device, torch.float16)
pixel_values = inputs.pixel_values

# generated_ids = model.generate(pixel_values, max_length=25)
generated_ids = model.generate(**inputs, max_length=25)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)